In [10]:
pip install cbbpy


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [11]:
pip install --upgrade cbbpy


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [8]:
import cbbpy.mens_scraper as s
import pandas as pd

# Scrape season 2026: info + boxscore only (no pbp)
game_info_df, boxscore_df, _ = s.get_games_season(
    2026,
    info=True,
    box=True,
    pbp=False
)

# Optional: quick sanity check
print("game_info_df:", game_info_df.shape)
print("boxscore_df:", boxscore_df.shape)

# Save to CSV
game_info_df.to_csv("game_info_2026.csv", index=False)
boxscore_df.to_csv("game_boxscores_2026.csv", index=False)

print("Saved: game_info_2026.csv and game_boxscores_2026.csv")

Scraping 15 games on 01/26/26: 100%|██████████| 87 of 87 days scraped in 1318.7 sec 


Log file is located at /Users/frankthaingtham/Library/Logs/CBBpy/2.1.2/CBBpy.log
game_info_df: (4016, 29)
boxscore_df: (91647, 24)
Saved: game_info_2026.csv and game_boxscores_2026.csv


In [12]:
import cbbpy.mens_scraper as s
import pandas as pd

YEAR = 2026
START = "2026-02-07"
END   = "2026-03-07"

# 1) Pull all games involving ACC teams for the season (no pbp)
info_df, box_df, _ = s.get_games_conference(
    "acc",
    YEAR,
    info=True,
    box=True,
    pbp=False
)

# 2) Filter to date window
info_df["game_day_dt"] = pd.to_datetime(info_df["game_day"], errors="coerce")
mask_dates = (info_df["game_day_dt"] >= START) & (info_df["game_day_dt"] <= END)

# 3) Filter to ACC conference games only (if column exists)
# (CBBpy's game_info includes is_conference per README examples)
if "is_conference" in info_df.columns:
    mask_conf = info_df["is_conference"] == True
else:
    # fallback: keep everything if the flag isn't present
    mask_conf = True

acc_conf_games = info_df[mask_dates & mask_conf].copy()

# 4) Subset boxscores to those games
acc_game_ids = set(pd.to_numeric(acc_conf_games["game_id"], errors="coerce").dropna().astype(int))
box_df["game_id"] = pd.to_numeric(box_df["game_id"], errors="coerce")
acc_conf_box = box_df[box_df["game_id"].isin(acc_game_ids)].copy()

# 5) Save
acc_conf_games.drop(columns=["game_day_dt"], errors="ignore").to_csv(
    "acc_game_info_2026_0207_to_0307.csv", index=False
)
acc_conf_box.to_csv(
    "acc_game_boxscores_2026_0207_to_0307.csv", index=False
)

print("ACC conference games:", acc_conf_games.shape)
print("ACC boxscore rows:", acc_conf_box.shape)
print("Saved: acc_game_info_2026_0207_to_0307.csv and acc_game_boxscores_2026_0207_to_0307.csv")

Log file is located at /Users/frankthaingtham/Library/Logs/CBBpy/2.1.2/CBBpy.log


TypeError: cannot unpack non-iterable NoneType object

In [7]:
#!/usr/bin/env python3
"""
Download all NCAA D1 games for 2024-25 season
Save full game_info DataFrame to CSV in same directory
"""

import sys
import pandas as pd
from pathlib import Path

try:
    import cbbpy.mens_scraper as s
except ImportError:
    try:
        from cbbpy import mens_scraper as s
    except ImportError:
        sys.exit("❌ Could not import cbbpy.mens_scraper. Please install cbbpy first.")

print("📡 Fetching 2024-25 season data from cbbpy...")
games_info, _, _ = s.get_games_season(2025, info=True, box=False, pbp=False)

output_file = Path("game_info_2025.csv").resolve()

games_info.to_csv(output_file, index=False)

print(f"\n✅ Done! Saved to: {output_file}")
print(f"📊 Total games downloaded: {len(games_info)}")
print("\n👀 Preview:")
print(games_info.head())

📡 Fetching 2024-25 season data from cbbpy...


No games on 05/01/25: 100%|██████████| 182 of 182 days scraped in 558.3 sec          


Log file is located at /Users/frankthaingtham/Library/Logs/CBBpy/2.1.2/CBBpy.log

✅ Done! Saved to: /Users/frankthaingtham/Repository /ncaa-basketball-spread/game_info_2025.csv
📊 Total games downloaded: 6299

👀 Preview:
     game_id game_status                  home_team  \
0  401727499       Final  IU Columbus Crimson Pride   
1  401715872       Final         Charleston Cougars   
2  401725796       Final    UNT Dallas Trailblazers   
3  401726400       Final  Oglethorpe Stormy Petrels   
4  401726395       Final          LaGrange Panthers   

                        home_id  home_rank home_record  home_score  \
0  nd-iu-columbus-crimson-pride        NaN                      44   
1                           232        NaN         1-0          90   
2    nd-unt-dallas-trailblazers        NaN                      55   
3  nd-oglethorpe-stormy-petrels        NaN                      51   
4          nd-lagrange-panthers        NaN                      48   

                       away_

In [7]:
#!/usr/bin/env python3
"""
Collect all D1 team names for 2024-25 via cbbpy.mens_scraper.get_teams_from_conference
- Saves: teams_2025_d1.csv (columns: conference, team)
- Uses EXACT NCAA canonical conference names (no fuzzy aliases)
- Writes CSV next to the script (or CWD in notebooks)
"""

import sys
from pathlib import Path
import pandas as pd

# Import cbbpy
try:
    import cbbpy.mens_scraper as s
except ImportError:
    try:
        from cbbpy import mens_scraper as s
    except ImportError:
        sys.exit("❌ Could not import cbbpy.mens_scraper. Please install cbbpy first.")

SEASON = 2025

# Canonical 2024–25 Division I conference names (32)
CANONICAL_CONFS = [
    "America East Conference",
    "American Athletic Conference",
    "Atlantic 10 Conference",
    "Atlantic Coast Conference",
    "ASUN Conference",
    "Big 12 Conference",
    "Big East Conference",
    "Big Sky Conference",
    "Big South Conference",
    "Big Ten Conference",
    "Big West Conference",
    "Coastal Athletic Association",
    "Conference USA",
    "Horizon League",
    "Ivy League",
    "Metro Atlantic Athletic Conference",
    "Mid-American Conference",
    "Mid-Eastern Athletic Conference",
    "Missouri Valley Conference",
    "Mountain West Conference",
    "Northeast Conference",
    "Ohio Valley Conference",
    "Patriot League",
    "Southeastern Conference",
    "Southern Conference",
    "Southland Conference",
    "Summit League",
    "Sun Belt Conference",
    "Southwestern Athletic Conference",
    "Western Athletic Conference",
    "West Coast Conference",
]

def main():
    rows, failures = [], []

    print("📡 Fetching D1 team lists by canonical conference names...")
    for conf_name in CANONICAL_CONFS:
        try:
            teams = s.get_teams_from_conference(conf_name, season=SEASON)
            if not teams:
                raise ValueError("No teams returned")
            print(f"  ✅ {conf_name} → {len(teams)} teams")
            for t in teams:
                rows.append({"conference": conf_name, "team": t})
        except Exception as e:
            failures.append((conf_name, str(e)))
            print(f"  ⚠️  {conf_name}: FAILED — {e}")

    if not rows:
        sys.exit("❌ No teams collected. Check cbbpy version or conference names.")

    df = pd.DataFrame(rows).drop_duplicates().sort_values(["conference", "team"])

    # Save next to script if __file__ exists; else current working directory (e.g., notebooks)
    try:
        script_dir = Path(__file__).parent.resolve()
    except NameError:
        script_dir = Path(".").resolve()

    out_path = script_dir / "teams_2025_d1.csv"
    df.to_csv(out_path, index=False)

    print("\n✅ Saved:", out_path)
    print("📊 Conferences succeeded:", df['conference'].nunique(), "of", len(CANONICAL_CONFS))
    print("👥 Unique teams:", df['team'].nunique())
    if failures:
        print("\n⚠️ Conferences that failed (double-check spelling or scraper support):")
        for name, err in failures:
            print(f"  - {name}: {err}")

    print("\n👀 Preview:")
    print(df.head(12).to_string(index=False))

if __name__ == "__main__":
    main()

📡 Fetching D1 team lists by canonical conference names...
  ✅ America East Conference → 9 teams
  ✅ American Athletic Conference → 13 teams
  ✅ Atlantic 10 Conference → 15 teams
  ✅ Atlantic Coast Conference → 18 teams
  ✅ ASUN Conference → 12 teams
  ✅ Big 12 Conference → 16 teams
  ✅ Big East Conference → 11 teams
  ✅ Big Sky Conference → 10 teams
  ✅ Big South Conference → 9 teams
  ✅ Big Ten Conference → 18 teams
  ✅ Big West Conference → 11 teams
  ✅ Coastal Athletic Association → 14 teams
  ✅ Conference USA → 10 teams
  ✅ Horizon League → 11 teams
  ✅ Ivy League → 8 teams
  ✅ Metro Atlantic Athletic Conference → 13 teams
  ✅ Mid-American Conference → 12 teams
  ✅ Mid-Eastern Athletic Conference → 8 teams
  ✅ Missouri Valley Conference → 12 teams
  ✅ Mountain West Conference → 11 teams
  ✅ Northeast Conference → 9 teams
  ✅ Ohio Valley Conference → 11 teams
  ✅ Patriot League → 10 teams
  ✅ Southeastern Conference → 16 teams
  ✅ Southern Conference → 10 teams
  ✅ Southland Confere